# Day 16: Aunt Sue

[*Advent of Code 2015 day 16*](https://adventofcode.com/2015/day/16) and [*solution megathread*](https://www.reddit.com/3x1i26)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/16/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F16%2Fcode.ipynb)

## Boilerplate

Since I improved my ability to type hint (which is probably a good thing even for program correctness), I realized pycodestyle doesn't give a heck about types - we need nb_mypy for that.

In [1]:
%load_ext pycodestyle_magic

In [2]:
%pycodestyle_on

In [3]:
%load_ext nb_mypy

Version 1.0.4


In [4]:
%nb_mypy On

In [5]:
import sys
sys.path.append('../../')
import common
import logging


logging.basicConfig(filename="debug.log", level=logging.WARNING)

downloaded = common.refresh()
%store downloaded >downloaded

3:1: E402 module level import not at top of file
4:1: E402 module level import not at top of file
10:20: E225 missing whitespace around operator


## Part One

In [6]:
from IPython.display import HTML


HTML(downloaded['part1'])

## Comments

Funny we didn't get any test data this time - but this ought to be quite doable with a bit of Pandas?

_Update:_ Actually, seems there were little point in using Pandas (though it was pleasant to parse the data into DataFrames), but I did do it using a decorator to decouple the code (including type hints), which took too much experimenting, but turned out much nicer than the naive attempt. Really, all these if/else criteria are really nasty to go at naively, I had so many little errors (the final one arrived at the right answer, though mistakingly included one "aunt" which ought to have been excluded)

In [7]:
from typing import Dict
from IPython.display import display


categories = """children
cats
samoyeds
pomeranians
akitas
vizslas
goldfish
trees
cars
perfumes""".splitlines()

traces_str = """children: 3
cats: 7
samoyeds: 2
pomeranians: 3
akitas: 0
vizslas: 0
goldfish: 5
trees: 3
cars: 2
perfumes: 1""".splitlines()

# This was created before we started nb_mypy
downloaded: Dict[str, str]
inputdata = downloaded['input'].splitlines()
display(f'{inputdata[0]} ...')

'Sue 1: cars: 9, akitas: 3, goldfish: 0 ...'

In [8]:
from typing import TypeVar, ParamSpec, Iterable, Tuple, List, Dict, DefaultDict
from collections.abc import Callable
from functools import wraps


Substances = Dict[str, int]
AuntsPosessions = Dict[int, Substances]


def parse_data(
        data: List[str],
        traces_str: List[str],
        categories: List[str]) -> Tuple[AuntsPosessions, Substances]:
    def value_to_int(k_v: Tuple[str, ...]) -> Tuple[str, int]:
        return k_v[0], int(k_v[1])

    def colon_value_tuple(item_s: str) -> Tuple[str, ...]:
        return tuple(item_s.split(': ', 1))

    P = ParamSpec("P")

    def assert_substances(fun: Callable[P, Substances]) -> \
            Callable[P, Substances]:
        @wraps(fun)
        def decorator(*args: P.args, **kwargs: P.kwargs) -> Substances:
            output = fun(*args, **kwargs)
            for key in output:
                assert key in categories, f'{key=} not in {categories=}'
            return output
        return decorator

    @assert_substances
    def colon_intvalues_dict(items_s: Iterable[str]) -> Substances:
        return dict(map(value_to_int,
                        map(colon_value_tuple,
                            items_s)))

    def colon_substr_dict_dict(prefix: str,
                               data: Iterable[str]) -> AuntsPosessions:
        output = dict()
        for line in data:
            auntid_s, items_s, *_ = colon_value_tuple(line)
            auntid = int(auntid_s[len('Sue '):])
            output[auntid] = colon_intvalues_dict(items_s.split(', '))
        return output

    return (colon_substr_dict_dict('Sue ', data),
            colon_intvalues_dict(traces_str))

In [9]:
aunts_posessions, traces = parse_data(inputdata, traces_str, categories)

In [10]:
eliminated_aunts = []
for substance, amount in traces.items():
    for aunt_id, known_posessions in aunts_posessions.items():
        if (
                substance in known_posessions and
                amount != known_posessions[substance]):
            eliminated_aunts.append(aunt_id)
            break
remaining_aunts = {aunt_id: known_posessions
                   for aunt_id, known_posessions in aunts_posessions.items()
                   if aunt_id not in eliminated_aunts}
for aunt_id, known_posessions in remaining_aunts.items():
    for substance, amount in known_posessions.items():
        if (
                substance not in traces or
                amount != traces[substance]):
            eliminated_aunts.append(aunt_id)
            break
remaining_aunts = {aunt_id: known_posessions
                   for aunt_id, known_posessions in aunts_posessions.items()
                   if aunt_id not in eliminated_aunts}
display(f'{remaining_aunts=}')

"remaining_aunts={373: {'pomeranians': 3, 'perfumes': 1, 'vizslas': 0}}"

In [11]:
from typing import Optional


# In particular, the cats and trees readings indicates that there are greater
# than that many (due to the unpredictable nuclear decay of cat dander and tree
# pollen), while the pomeranians and goldfish readings indicate that there are
# fewer than that many (due to the modial interaction of magnetoreluctance).
READING_IS_LOWER_BOUND = ['cats', 'trees']
READING_IS_UPPER_BOUND = ['pomeranians', 'goldfish']


def process_of_elimination(
        fun: Callable[..., List[int]]) -> \
        Callable[..., AuntsPosessions]:
    @wraps(fun)
    def decoration(
            traces: Substances,
            aunts_posessions: AuntsPosessions,
            modified: Optional[bool] = False) -> \
            AuntsPosessions:
        eliminated = fun(
            traces=traces,
            aunts_posessions=aunts_posessions,
            modified=modified)
        output = {
            aunt_id: posessions
            for aunt_id, posessions in aunts_posessions.items()
            if aunt_id not in eliminated}
        logging.debug(f'from {len(aunts_posessions)}, eliminated ' +
                      f'{len(eliminated)}, resulting in {len(output)}')
        return output
    return decoration


def trace_contradicts_possessed(substance: str,
                                amount_trace: int,
                                amount_posession: int,
                                modified: Optional[bool]) -> bool:
    output: bool
    if modified and substance in READING_IS_LOWER_BOUND:
        output = amount_trace >= amount_posession
    elif modified and substance in READING_IS_UPPER_BOUND:
        output = amount_trace <= amount_posession
    else:
        output = amount_trace != amount_posession
    logging.debug(f"trace {substance} ({amount_trace}) contradicts " +
                  f"{amount_posession} ({modified=}): {output}")
    return output


def trace_contradicts_aunt(name_trace: str,
                           amount_trace: int,
                           posessions: Substances,
                           modified: Optional[bool] = False) -> bool:
    try:
        return trace_contradicts_possessed(
            substance=name_trace,
            amount_trace=amount_trace,
            amount_posession=posessions[name_trace],
            modified=modified)
    # If there existed a trace which we do not know
    # aunt posessed, we cannot eliminate her
    except KeyError:
        logging.debug(f"We don't know aunt had any {name_trace}")
        return False


@process_of_elimination
def eliminate_per_trace(
        traces: Substances,
        aunts_posessions: AuntsPosessions,
        modified: Optional[bool] = False) -> List[int]:
    eliminated = []
    for name_trace, amount_trace in traces.items():
        for auntid, posessions in aunts_posessions.items():
            if auntid in eliminated:
                break

            if trace_contradicts_aunt(
                    name_trace=name_trace,
                    amount_trace=amount_trace,
                    posessions=posessions,
                    modified=modified):
                logging.debug(f"eliminated {auntid} due to {name_trace}")
                eliminated.append(auntid)
    return eliminated


def posession_contradict_traces(
        name_posession: str,
        amount_posession: int,
        traces: Substances,
        modified: Optional[bool] = False) -> bool:
    try:
        return trace_contradicts_possessed(
            substance=name_posession,
            amount_trace=traces[name_posession],
            amount_posession=amount_posession,
            modified=modified)
    # If aunt posessed anything which was not traced, eliminate her
    except KeyError:
        logging.debug(f"Aunt had {name_posession} which there " +
                      "was no trace of")
        return True


@process_of_elimination
def eliminate_per_posession(
        traces: Substances,
        aunts_posessions: AuntsPosessions,
        modified: Optional[bool] = False) -> List[int]:
    eliminated = []
    for auntid, posessions in aunts_posessions.items():
        for name_posession, amount_posession in posessions.items():
            if posession_contradict_traces(
                    name_posession=name_posession,
                    amount_posession=amount_posession,
                    traces=traces,
                    modified=modified):
                logging.debug(f"eliminated {auntid} due to {name_posession}")
                eliminated.append(auntid)
                break
    return eliminated

In [12]:
def my_part1_solution(
        traces: Substances,
        aunts_posessions: AuntsPosessions) -> int:
    remaining_aunts = eliminate_per_trace(
        traces=traces,
        aunts_posessions=aunts_posessions,
        modified=False)
    remaining_aunts = eliminate_per_posession(
        traces=traces,
        aunts_posessions=remaining_aunts,
        modified=False)
    if len(remaining_aunts) != 1:
        raise ValueError(f'Failed to find a single aunt: {remaining_aunts=}')
    return list(remaining_aunts)[0]

In [13]:
display(my_part1_solution(traces, aunts_posessions))

373

In [14]:
HTML(downloaded['part1_footer'])

## Part Two

In [15]:
HTML(downloaded['part2'])

In [16]:
debug = True
eliminated_aunts = []
for substance, amount in traces.items():
    for auntid, known_posessions in aunts_posessions.items():
        if (substance in known_posessions and (
                (substance in READING_IS_LOWER_BOUND and
                 amount >= known_posessions[substance]) or
                (substance in READING_IS_UPPER_BOUND and
                 amount <= known_posessions[substance]) or
                amount != known_posessions[substance])):
            eliminated_aunts.append(auntid)
            logging.debug(
                f"Eliminated aunt {auntid} due to having wrong amount " +
                f"{known_posessions[substance]} of {substance} " +
                f"(expected {amount})")
            break
if debug:
    display(f'{eliminated_aunts=}')
remaining_aunts = {auntid: known_posessions
                   for auntid, known_posessions in aunts_posessions.items()
                   if auntid not in eliminated_aunts}
eliminated_aunts = []
for auntid, known_posessions in remaining_aunts.items():
    for substance, amount in known_posessions.items():
        if (
                substance not in traces or
                (substance not in READING_IS_LOWER_BOUND and
                 amount > traces[substance]) or
                (substance not in READING_IS_UPPER_BOUND and
                 amount < traces[substance]) or
                ((substance in READING_IS_LOWER_BOUND or
                  substance in READING_IS_UPPER_BOUND) and
                 amount == traces[substance])):
            if substance not in traces:
                logging.debug(
                    f"Eliminated {auntid} due to not finding any trace" +
                    f"of {substance} (expected {amount})")
            else:
                logging.debug(
                    f"Eliminated aunt {auntid} due to not finding right " +
                    f"amount of {substance} ({traces[substance]}) " +
                    f"(expected {amount})")
            eliminated_aunts.append(auntid)
            break
if debug:
    display(f'{eliminated_aunts=}')
remaining_aunts = {auntid: known_posessions
                   for auntid, known_posessions in remaining_aunts.items()
                   if auntid not in eliminated_aunts}
display(f'{remaining_aunts=}')

'eliminated_aunts=[3, 13, 2, 7, 1, 4, 1, 3, 1, 7]'

'eliminated_aunts=[5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222

"remaining_aunts={260: {'goldfish': 0, 'vizslas': 0, 'samoyeds': 2}}"

In [17]:
def my_part2_solution(
        traces: Substances,
        aunts_posessions: AuntsPosessions,
        debug: Optional[bool] = False) -> int:
    remaining_aunts = eliminate_per_trace(
        traces=traces,
        aunts_posessions=aunts_posessions,
        modified=True)
    remaining_aunts = eliminate_per_posession(
        traces=traces,
        aunts_posessions=remaining_aunts,
        modified=True)
    if len(remaining_aunts) != 1:
        raise ValueError(f'Failed to find a single aunt: {remaining_aunts=}')
    return list(remaining_aunts)[0]

In [18]:
display(my_part2_solution(traces, aunts_posessions, True))

260

In [19]:
HTML(downloaded['part2_footer'])